In [ ]:
import cv2
import itertools as it
import numpy as np
import pickle

def show_and_wait(frame, title='tesst', wait_time=0):
    cv2.imshow(title, frame)
    key = cv2.waitKey(wait_time)
    return None if key < 0 else chr(key)

def read_and_wait(video, title='tesst', wait_time=0):
    result, frame = video.read()
    if result:
        return show_and_wait(frame, title, wait_time)

In [ ]:
file_path = r"C:\Users\cidzerda\Documents\GitHub\ctc\hero_names.txt"
with open(file_path) as fin:
    hero_names = [s.strip() for s in fin]
len(hero_names)

In [ ]:
all_letters = set(chr(i + ord('A')) for i in range(26))
def get_letters(hero_names):
    g = it.chain.from_iterable(hero_names)
    s = set(c for c in g if 'A' <= c <= 'Z')
    return all_letters - s
def get_letter(letters, available_hero_names):
    def count_names(c):
        l = [s for s in available_hero_names if c in s]
        return len(l)
    d = {count_names(c): c for c in letters}
    n = min(d)
    return d[n]
def get_names(letter, available_hero_names):
    return [s for s in available_hero_names if letter in s]
def get_name(l, letters):
    def count_letters(s):
        s = letters - set(s)
        return len(s)
    d = {count_letters(s): s for s in l}
    n = min(d)
    return d[n]
def fn(hero_names):
    # Create a list of selected hero names.
    selected_hero_names = []
    # Create a list of available hero names.
    available_hero_names = list(hero_names)
    # Create a set of accounted letters.
    accounted_letters = set()
    while len(accounted_letters) < 26:
        # Create a list of letters that are not represented in the list of selected names.
        letters = all_letters - accounted_letters
        # From that list, select the letter that is in the fewest names of those in the available names.
        letter = get_letter(letters, available_hero_names)
        # Get the list of names that contain that letter and are in the available names.
        l = get_names(letter, available_hero_names)
        # From that list, select the name with the greatest number of letters that are
        # not represented in the list of selected names.
        s = get_name(l, letters)
        # Add that name to the list of selected names.
        selected_hero_names.append(s)
        # Add that name's letters to the set of accounted letters.
        accounted_letters.update(c for c in s if 'A' <= c <= 'Z')
    return selected_hero_names
print(fn(hero_names))

In [ ]:
video = cv2.VideoCapture(r"C:\Users\cidzerda\Documents\GitHub\strevr\data\tmp\2020-01-15-grandgrant.mp4")
print(video.get(cv2.CAP_PROP_FRAME_COUNT))
video.set(cv2.CAP_PROP_POS_MSEC, (19 * 60 + 21) * 1000) # game start
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (22 * 60 + 56) * 1000) # before UI change
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (23 * 60 + 15) * 1000) # after UI change
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (25 * 60 +  0) * 1000) # before pause
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (25 * 60 + 50) * 1000) # after pause
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (27 * 60 + 59) * 1000) # before UI change
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (28 * 60 + 11) * 1000) # after UI change
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (53 * 60 + 34) * 1000) # game end
read_and_wait(video)
video.release()
cv2.destroyAllWindows()

In [ ]:
ranges = (
    (19 * 60 + 21, 22 * 60 + 56),
    (23 * 60 + 15, 25 * 60 +  0),
    (25 * 60 + 50, 27 * 60 + 59),
    (28 * 60 + 11, 53 * 60 + 34),
)
video = cv2.VideoCapture(r"C:\Users\cidzerda\Documents\GitHub\strevr\data\2020-01-15-grandgrant.mp4")
for begin, end in ranges:
    print(begin)
    for i in range(begin, end):
        video.set(cv2.CAP_PROP_POS_MSEC, i * 1000)
        _, frame = video.read()
        ch = show_and_wait(frame, wait_time=25)
        if ch == 'q':
            break
        if ch == ' ':
            print(video.get(cv2.CAP_PROP_POS_MSEC))
            input()
video.release()
cv2.destroyAllWindows()

In [ ]:
ranges = (
    (19 * 60 + 21, 22 * 60 + 56),
    (23 * 60 + 15, 25 * 60 +  0),
    (25 * 60 + 50, 27 * 60 + 59),
    (28 * 60 + 11, 53 * 60 + 34),
)
print(sum(end - begin for begin, end in ranges))

In [ ]:
with open(r"C:\Users\cidzerda\Documents\GitHub\strevr\data\selected.txt") as fin:
    labels =  fin.readline()
with open(r"C:\Users\cidzerda\Documents\GitHub\strevr\data\selected.pickle", 'rb') as fin:
    images = [pickle.load(fin) for _ in labels]

# Move invalid images to a separate array.
invalid_images = images[-58:-54] + images[-45:-41] + images[-11:] # There are nineteen of these.
del images[-58:-54]
del images[-45:-41]
del images[-11:]

# Map all images to L*u*v*.
luvs = [cv2.cvtColor(image, cv2.COLOR_BGR2Luv) for image in images]

# Combine the list of three-dimensional tensors into a single four-dimensional tensor.
images = np.stack(images, axis=0)
luvs = np.stack(luvs, axis=0)

# There are 2981 valid images.  That number's prime factors are 11 and 271.
print(images.shape)

In [ ]:
def fn(desired_ells, desired_factor, color):
    background = np.ones(desired_factor.shape, dtype=np.float32)
    background = np.stack([c * background for c in color], axis=-1)
    luv = cv2.cvtColor(background, cv2.COLOR_BGR2Luv)
    luv[:,:,0] = luv[:,:,0] * (1.0 - desired_factor) + desired_ells * desired_factor
    image = cv2.cvtColor(luv, cv2.COLOR_Luv2BGR)
    return image

# Extract all L*.
ells = luvs[:,:,:,0]

# Determine the desired L* by computing the mean of all L*.
mean = np.mean(ells, axis=0)
desired_ells = mean

# Determine the desired factor by scaling the range of standard deviations of all L* from [min, max] to [1, 0].
std = np.std(ells, axis=0)
desired_factor = (std - np.max(std)) / (np.min(std) - np.max(std))

# Display examples side-by-side.
random_indices = sorted(np.random.uniform(0, len(images), [9]).astype(int))
random_indices = [99, 558, 636, 1201, 2012, 2543, 2694, 2781, 2917] # These are good.
g = ((fn(desired_ells, desired_factor, images[i][3, 17]), images[i]) for i in random_indices)
l = list(it.chain.from_iterable(g))
show_and_wait(cv2.resize(np.vstack(l), (520, 60 * len(l))))

cv2.destroyAllWindows()